In [ ]:
import numpy as np

class LogisticRegressionOvR:
    """
    One-vs-Rest Logistic Regression using Gradient Descent.
    """

    def __init__(self, learning_rate=0.01, epochs=1000, tol=1e-8, verbose=False):
        self.learning_rate = learning_rate
        self.epochs = epochs
        self.tol = tol
        self.verbose = verbose
        self.weights = None      # shape: (n_features+1, n_classes)
        self.loss_history = []
        self.n_classes = None

    def add_bias(self, X):
        """Add bias column (intercept term)."""
        return np.hstack((np.ones((X.shape[0], 1)), X))

    def sigmoid(self, z):
        return 1 / (1 + np.exp(-z))

    def _binary_loss(self, y_true, y_pred):
        """Binary cross-entropy for one classifier."""
        eps = 1e-9
        return -np.mean(y_true*np.log(y_pred+eps) + (1-y_true)*np.log(1-y_pred+eps))

    def fit(self, X, Y):
        """
        Train K binary classifiers (one per class).
        
        Parameters
        ----------
        X : np.ndarray of shape (n_samples, n_features)
        Y : np.ndarray of shape (n_samples,) with integer labels
        """
        X = np.asarray(X, dtype=np.float64)
        Y = np.asarray(Y, dtype=np.int64)

        self.n_classes = len(np.unique(Y))
        X = self.add_bias(X)
        m, n = X.shape

        # Initialize weights for K classifiers
        self.weights = np.zeros((n, self.n_classes))
        self.loss_history = [[] for _ in range(self.n_classes)]

        for c in range(self.n_classes):
            # Binary labels for class c vs rest
            y_c = (Y == c).astype(np.float64).reshape(-1, 1)
            w_c = np.zeros((n, 1))

            prev_loss = float("inf")
            for epoch in range(self.epochs):
                preds = self.sigmoid(X @ w_c)

                # Gradient
                grad = (X.T @ (preds - y_c)) / m
                w_c -= self.learning_rate * grad

                # Loss
                loss = self._binary_loss(y_c, preds)
                self.loss_history[c].append(loss)

                # Early stopping
                if abs(prev_loss - loss) < self.tol:
                    break
                prev_loss = loss

            self.weights[:, c] = w_c.ravel()

            if self.verbose:
                print(f"Trained classifier for class {c}, final loss={loss:.6f}")

        return self

    def predict_proba(self, X):
        """Predict probabilities for each class (OvR)."""
        X = np.asarray(X, dtype=np.float64)
        X = self.add_bias(X)
        probs = self.sigmoid(X @ self.weights)
        return probs

    def predict(self, X):
        """Predict class labels (argmax over OvR probabilities)."""
        return np.argmax(self.predict_proba(X), axis=1)

    def score(self, X, Y):
        """Compute accuracy."""
        preds = self.predict(X)
        return np.mean(preds == Y)

In [ ]:
# Fake dataset: 3 classes, 2 features
np.random.seed(0)
X = np.random.randn(200, 2)
Y = np.random.choice(3, 200)  # labels in {0,1,2}

# Train OvR Logistic Regression
model = LogisticRegressionOvR(learning_rate=0.1, epochs=1000, verbose=True)
model.fit(X, Y)

# Predictions
print("Predicted labels:", model.predict(X[:5]))
print("Predicted probabilities:\n", model.predict_proba(X[:5]))
print("Accuracy:", model.score(X, Y))